# Тестовые данные

In [197]:
import csv


questions = [
    "Что такое курс рубля и почему он меняется?",
    "Зачем Банк России перешел к режиму плавающего курса?",
    "Проводит ли Банк России операции с целью укрепить или ослабить рубль?"
]

answers = [
    """
    Курс рубля к иностранной валюте - это стоимость одного рубля, 
    выраженная в единицах иностранной валюты.
    Курс рубля меняется под влиянием спроса и предложения на 
    иностранную валюту на валютном рынке.

    На курс рубля могут влиять:
    - Объемы внешней торговли
    - Импортные и экспортные цены
    - Уровни инфляции и процентных ставок
    - Темпы экономического роста
    - Периоды нестабильности
    - Изменения денежно-кредитной политики
    """,
    """
    Банк России перешел к режиму плавающего курса в 2014 году.
    Плавающий курс позволяет:
    - Проводить самостоятельную денежно-кредитную политику
    - Снижать инфляцию
    - Экономике подстраиваться под меняющиеся внешние условия
    """,
    """
    В обычных условиях Банк России не вмешивается в курс рубля.
    Банк России может проводить операции на валютном рынке:
    - Для поддержания финансовой стабильности
    - Для пополнения международных резервов
    - В 2014 году Банк России проводил интервенции для поддержки рубля
    """
]


links = [
    "https://www.cbr.ru/about_br/publ/ddkp/",
    "https://www.cbr.ru/dkp/exchange_rate/",
    "https://cbr.ru/oper_br/o_a/currency/"
]


with open("test.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["answer", "link"])
    for answer, link in zip(answers, links):
        writer.writerow([answer, link])




In [230]:
import pandas as pd
test_data= pd.read_csv("test.csv") 
test_data["answer"] = test_data["answer"].str.replace("\n", " ")

In [231]:
test_data['answer'][0]

'     Курс рубля к иностранной валюте - это стоимость одного рубля,      выраженная в единицах иностранной валюты.     Курс рубля меняется под влиянием спроса и предложения на      иностранную валюту на валютном рынке.      На курс рубля могут влиять:     - Объемы внешней торговли     - Импортные и экспортные цены     - Уровни инфляции и процентных ставок     - Темпы экономического роста     - Периоды нестабильности     - Изменения денежно-кредитной политики     '

# Эмбеддинги

In [200]:
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import torch

In [232]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings = model.encode(test_data['answer'])
print(embeddings)

[[ 0.01131858 -0.16646418 -0.01336718 ... -0.02108256  0.02286251
   0.03454603]
 [ 0.03172838 -0.17987297 -0.0171023  ...  0.01832074 -0.07802644
   0.04500177]
 [ 0.10769528 -0.19871435 -0.01545429 ...  0.04401986 -0.05108445
   0.04408291]]


In [233]:
embeddings.shape

(3, 768)

In [234]:
pca = Pipeline(steps=[
    ('mean', StandardScaler(with_mean=True, with_std=False)),
    ('pca', PCA(n_components=2, random_state=42)),
    ('std', StandardScaler(with_mean=True, with_std=True))
])
pca.fit(embeddings)

Pipeline(steps=[('mean', StandardScaler(with_std=False)),
                ('pca', PCA(n_components=2, random_state=42)),
                ('std', StandardScaler())])

In [235]:
text_vectors = pca.transform(embeddings)
text_vectors.shape

(3, 2)

In [205]:
text_vectors

array([[ 1.4124194 , -0.07120789],
       [-0.64454186,  1.2587956 ],
       [-0.76787776, -1.1875875 ]], dtype=float32)

# ClickHouse

In [213]:
# Создание таблицы
import clickhouse_driver

client = clickhouse_driver.Client(host='localhost')
client.execute("""
CREATE TABLE embeddings (
    id UInt32,
    embedding Array(Float32),
    link String
)
ENGINE = MergeTree
PRIMARY KEY (id)
""")

[]

In [236]:
client.execute('SHOW TABLES')

[('embeddings',), ('my_first_table',)]

In [215]:
#Заполнение таблицы
import clickhouse_connect

client = clickhouse_connect.get_client()

embeddings = text_vectors.tolist()
links = data['link'].tolist()
ids = list(range(len(text_vectors)))

data = []

for i in range(len(ids)):
    row = [ids[i], embeddings[i], links[i]]
    data.append(row)

client.insert('embeddings', data, column_names=['id', 'embedding', 'link'])

In [216]:
#Вывод
import clickhouse_driver

client = clickhouse_driver.Client(host='localhost')
client.execute("SELECT * FROM embeddings")

[(0,
  [1.4124194383621216, -0.07120788842439651],
  'https://www.cbr.ru/about_br/publ/ddkp/'),
 (1,
  [-0.64454185962677, 1.2587956190109253],
  'https://www.cbr.ru/dkp/exchange_rate/'),
 (2,
  [-0.7678777575492859, -1.1875874996185303],
  'https://cbr.ru/oper_br/o_a/currency/')]

In [217]:
result = client.execute("SELECT * FROM embeddings")
result_df = pd.DataFrame(result)

result_df

,0,1,2
0,0,"[1.4124194383621216, -0.07120788842439651]",https://www.cbr.ru/about_br/publ/ddkp/
1,1,"[-0.64454185962677, 1.2587956190109253]",https://www.cbr.ru/dkp/exchange_rate/
2,2,"[-0.7678777575492859, -1.1875874996185303]",https://cbr.ru/oper_br/o_a/currency/


In [207]:
#client.execute("DROP TABLE embeddings")

[]

In [241]:
sentence = [questions[2]]
sentence

['Проводит ли Банк России операции с целью укрепить или ослабить рубль?']

In [242]:
embedding = model.encode(sentence)
pca_embedding = pca.transform(embedding)
pca_embedding = np.squeeze(pca_embedding)
pca_embedding.tolist()

[-0.7126147747039795, -0.6340107917785645]

In [245]:
# cosineDistance or L2Distance
# Найти n ближайших соседа

query_vector = pca_embedding.tolist()

results = client.execute("""
SELECT id, embedding, link
FROM embeddings
ORDER BY L2Distance(embedding, {}) ASC
LIMIT 1
""".format(query_vector))


for row in results:
    id, embedding, link = row
    print(f"ID: {id}, Link: {link}, Answer: {test_data['answer'][id]}")


ID: 2, Link: https://cbr.ru/oper_br/o_a/currency/, Answer:      В обычных условиях Банк России не вмешивается в курс рубля.     Банк России может проводить операции на валютном рынке:     - Для поддержания финансовой стабильности     - Для пополнения международных резервов     - В 2014 году Банк России проводил интервенции для поддержки рубля     
